In [111]:
#%%
from array import array
from cmath import nan
from pyexpat import model
import statistics
from tkinter.ttk import Separator
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms
from torch.autograd import variable
from itertools import chain
from sklearn import metrics as met
import pickle
from icecream import ic

import matplotlib.pyplot as plt
import pathlib
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
# import util
# import model_torch_simple
# from torchmetrics import Accuracy
from tqdm import tqdm
import argparse
from icecream import ic
import numpy as np
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(42)


In [2]:
def value_counts_list(lst):
    """
    Computes the frequency count of unique elements in a list and returns a dictionary, sorted by frequency count in
    descending order.

    Args:
    - lst (list): List of elements

    Returns:
    - dict: Dictionary with unique elements as keys and their frequency count as values, sorted by frequency count
    in descending order
    """
    value_counts = {}
    for item in lst:
        if item in value_counts:
            value_counts[item] += 1
        else:
            value_counts[item] = 1
    sorted_value_counts = dict(sorted(value_counts.items(), key=lambda x: x[1], reverse=True))
    return sorted_value_counts

def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

### Loading data

In [3]:
train_data = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/gene_seq_train.csv')
train_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/res_train.csv')
#don't touch test data, split out validation data from training data during training
test_data = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/gene_seq_test.csv')
test_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/res_test.csv')

In [4]:
N_samples = train_data.shape[0]
DRUGS = ['AMIKACIN',
 'CAPREOMYCIN',
 'CIPROFLOXACIN',
 'ETHAMBUTOL',
 'ETHIONAMIDE',
 'ISONIAZID',
 'KANAMYCIN',
 'LEVOFLOXACIN',
 'MOXIFLOXACIN',
 'OFLOXACIN',
 'PYRAZINAMIDE',
 'RIFAMPICIN',
 'STREPTOMYCIN']

DRUGS = train_target.columns
LOCI = train_data.columns
assert set(DRUGS) == set(train_target.columns)
N_drugs = len(DRUGS)

# Feature importance

In [ ]:

def one_hot_torch(seq: str, dtype=torch.int8):
    seq_bytes = torch.ByteTensor(list(bytes(seq, "utf-8")))
    acgt_bytes = torch.ByteTensor(list(bytes("ACGT", "utf-8")))
    arr = torch.zeros(4, (len(seq_bytes)), dtype=dtype)
    arr[0, seq_bytes == acgt_bytes[0]] = 1
    arr[1, seq_bytes == acgt_bytes[1]] = 1
    arr[2, seq_bytes == acgt_bytes[2]] = 1
    arr[3, seq_bytes == acgt_bytes[3]] = 1
    return arr

# def one_hot_torch(seq):
#     oh = []
#     for sample in seq:
#         sample = torch.ByteTensor(list(bytes(sample, "utf-8")))
#         acgt_bytes = torch.ByteTensor(list(bytes("ACGT", "utf-8")))
#         arr = torch.zeros((len(sample), 4), dtype=torch.int8)
#         arr[sample == acgt_bytes[0], 0] = 1
#         arr[sample == acgt_bytes[1], 1] = 1
#         arr[sample == acgt_bytes[2], 2] = 1
#         arr[sample == acgt_bytes[3], 3] = 1
#         oh.append(arr)
#     return torch.stack(oh)

def my_padding(seq_tuple):
    list_x_ = list(seq_tuple)
    max_len = len(max(list_x_, key=len))
    for i, x in enumerate(list_x_):
        list_x_[i] = x + "N"*(max_len-len(x))
    return list_x_

#! faster than my_padding try to incorporate
def collate_padded_batch(batch):
    # get max length of seqs in batch
    max_len = max([x[0].shape[1] for x in batch])
    return torch.utils.data.default_collate(
        [(F.pad(x[0], (0, max_len - x[0].shape[1])), x[1]) for x in batch] #how does F.pad work
    )



def get_masked_loss(loss_fn):
    """
    Returns a loss function that ignores NaN values
    """

    def masked_loss(y_true, y_pred):
        y_pred = y_pred.view(-1, 13)  # Ensure y_pred has the same shape as y_true and non_nan_mask
        # ic(y_pred.shape)
        # ic(y_true.shape)
        non_nan_mask = ~y_true.isnan()
        # ic(non_nan_mask)
        y_true_non_nan = y_true[non_nan_mask]
        y_pred_non_nan = y_pred[non_nan_mask]
        # ic(y_true)
        # ic(y)
        # ic(y_true_non_nan)
        # ic(y_pred_non_nan)
        return loss_fn(y_pred_non_nan, y_true_non_nan)

    return masked_loss

masked_MSE = get_masked_loss(torch.nn.MSELoss())


# Dateset

In [151]:
# Julian's code - implement this, might be faster
class OneHotSeqsDataset(torch.utils.data.Dataset): #? what's the difference between using inheritance and not?
    def __init__(
        self,
        seq_df,
        res_df,
        target_loci=LOCI,
        target_drugs=DRUGS,
        one_hot_dtype=torch.int8,
    ):
        self.seq_df = seq_df[target_loci]
        self.res_df = res_df[target_drugs]
        if not self.seq_df.index.equals(self.res_df.index):
            raise ValueError(
                "Indices of sequence and resistance dataframes don't match up"
            )
        self.one_hot_dtype = one_hot_dtype

    def __getitem__(self, index):
        """
        numerical index --> get `index`-th sample
        string index --> get sample with name `index`
        """
        if isinstance(index, int):
            seqs_comb = self.seq_df.iloc[index].str.cat()
            res = self.res_df.iloc[index]
        elif isinstance(index, str):
            seqs_comb = self.seq_df.loc[index].str.cat()
            res = self.res_df.loc[index]
        else:
            raise ValueError(
                "Index needs to be an integer or a sample name present in the dataset"
            )
        return one_hot_torch(seqs_comb, dtype=self.one_hot_dtype), torch.tensor(res)

    def __len__(self):
        return self.res_df.shape[0]
    
training_dataset = OneHotSeqsDataset(train_data, train_target, one_hot_dtype=torch.float)
train_dataset, val_dataset = random_split(training_dataset, [int(len(training_dataset)*0.8), len(training_dataset)-int(len(training_dataset)*0.8)])

In [41]:
# class seq_dataset(Dataset):
#     def __init__(self, x, y):
#         self.x = x
#         self.y = y
#     def __getitem__(self, index):
#         seqs_comb = self.x.iloc[index,1:].str.cat(sep='X'*30)
#         seqs_comb = one_hot_torch(seqs_comb)
#         seqs_comb = seqs_comb.permute(2, 0, 1).contiguous().view(4, 57350)
#         res = torch.as_tensor(self.y.iloc[index,:].values.tolist())
        
#         return seqs_comb, res
#     def __len__(self):
#         return len(self.x)

# training_dataset = seq_dataset(train_data, train_target)
# train_dataset, val_dataset = random_split(training_dataset, [int(len(training_dataset)*0.8), len(training_dataset)-int(len(training_dataset)*0.8)])

# Model

In [167]:
torch.cuda.empty_cache()

model = Model(in_channel = 4, hidden_channel = 6, out_channel=13, batch_size=batch_size)
model = model.float()
model = model.to(device)

epoch = 20
batch_size = 8
lr = 0.001

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_padded_batch)
test_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, collate_fn=collate_padded_batch)
# criterion = nn.MSELoss()
criterion = masked_MSE
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


In [179]:
class Model(nn.Module):
    def __init__(self, in_channel = 4, hidden_channel = 64, out_channel=13, batch_size=1):
        super(Model, self).__init__()
        self.batch_size = batch_size
        # self.feature_extraction = nn.Conv1d(in_channels, hidden, kernel_size=kernel_size),
        self.conv1 = nn.Conv1d(in_channel, hidden_channel, kernel_size=25, stride=1)
        self.conv2 = nn.Conv1d(hidden_channel, hidden_channel, kernel_size=25, stride=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc = None
    def forward(self, x):
        x = F.relu(self.conv1(x))
        # ic('c1',x.size())
        x = F.relu(self.conv2(x))
        # ic('c2',x.size())
        
        x = F.relu(self.pool(x))
        # ic('pool',x.size()) 
        x = torch.max(x, dim=1).values
        # ic('max',x.size())
        # print('x after maxpool:', x.size())
        x = x.view(x.size(0), -1)
        # print('x after view:', x.size())

        if self.fc is None:
            # Set the input size based on x the first time forward is called
            # x = x.T
            input_size = x.size(1)
            self.fc = nn.Linear(input_size, 13).to(x.device)  # Ensure the layer is on the same device as x
        # print(x.size())

        x = torch.sigmoid(self.fc(x))
        x = torch.squeeze(x)
        # print('output:', x)
        # print('1111')             
        # first_dim_size = x.size(0)
        # x = x.reshape(first_dim_size, -1).contiguous()
        # first_dim_size = x.size(0)
        # print('size after research', x.size())
        return x

model = Model(in_channel = 4, hidden_channel = 8, out_channel=13, batch_size=batch_size)
model = model.float()
model = model.to(device)


# Training

In [180]:

ic.disable()
# ic.enable()
train_epoch_loss = []
test_epoch_loss = []

for e in tqdm(range(1, epoch+1)):
    model.train()
    train_batch_loss = []
    test_batch_loss = []
    for x, y in train_loader:
        x_batch = torch.squeeze(x, 0).to(device)
        y_batch = y.to(device)
        x_batch = x_batch.float()
        y_batch = y_batch.float()
        # y_batch = y_batch.view(-1)

        # y_batch = one_hot_torch(y).to(device)
        # print('batch y size before flatten:',y_batch.size())
        # y_batch = y_batch.flatten()
        # print('batch y size after flatten:',y_batch.size())
        # print(x_batch.size())
        # print(x_batch.size())
# For example, if you have a convolutional layer with 64 output channels, 3 input channels, and a kernel size of 3x3, the weight parameters would have a dimension of (64, 3, 3, 3)
        # print(x_batch.size())
        pred = model(x_batch.float())
        # print(x_batch)
        # print(pred)
        # pred = pred.unsqueeze(0)
        # ic(pred)
        # ic(y_batch)
        loss_train = criterion(y_batch, pred)
        ic(loss_train)
        train_batch_loss.append(loss_train)
        loss_train.backward()
        optimizer.step()
        optimizer.zero_grad()

    train_epoch_loss.append(torch.mean(torch.stack(train_batch_loss)).detach().cpu().numpy())
    with torch.no_grad():
        # print('test')
        for x, y in test_loader:
            x_batch = x.to(device)
            y_batch = y.to(device)
            # print(x_batch.size())
            # y_batch = torch.Tensor.float(y).to(device)
            # x_batch = x_batch.permute(0, 3, 1, 2).to(device)
            pred = model(x_batch.float())
            # pred = pred.unsqueeze(0)

            loss_test = criterion(y_batch, pred)
            test_batch_loss.append(loss_test)
        test_epoch_loss.append(torch.mean(torch.stack(test_batch_loss)).detach().cpu().numpy())
    print(f'Epoch {e}')
    print(f"Training loss: {torch.mean(torch.stack(train_batch_loss)).detach().cpu().numpy()}")
    print(f"Validation loss: {torch.mean(torch.stack(test_batch_loss)).detach().cpu().numpy()}") 
    # print(train_batch_loss)
    # print(test_batch_loss)
    # print(f"Training loss: {np.mean(train_batch_loss)}")
    # print(f"Validation loss: {np.mean(test_batch_loss)}")
    print('==='*10)

  0%|          | 0/20 [00:00<?, ?it/s]

In [174]:
t = torch.tensor([[0.5542, 0.4820, 0.4521, 0.4912, 0.5167, 0.5015, 0.4697, 0.4547, 0.4846,
                     0.5677, 0.4784, 0.4777, 0.5017],
                    [0.5542, 0.4820, 0.4521, 0.4912, 0.5167, 0.5015, 0.4697, 0.4547, 0.4846,
                     0.5677, 0.4784, 0.4777, 0.5017],
                    [0.5542, 0.4820, 0.4521, 0.4912, 0.5167, 0.5015, 0.4697, 0.4547, 0.4846,
                     0.5677, 0.4784, 0.4777, 0.5017],
                    [0.5542, 0.4820, 0.4521, 0.4912, 0.5167, 0.5015, 0.4697, 0.4547, 0.4846,
                     0.5677, 0.4784, 0.4777, 0.5017],
                    [0.5542, 0.4820, 0.4521, 0.4912, 0.5167, 0.5015, 0.4697, 0.4547, 0.4846,
                     0.5677, 0.4784, 0.4777, 0.5017],
                    [0.5542, 0.4820, 0.4521, 0.4912, 0.5167, 0.5015, 0.4697, 0.4547, 0.4846,
                     0.5677, 0.4784, 0.4777, 0.5017],
                    [0.5542, 0.4820, 0.4521, 0.4912, 0.5167, 0.5015, 0.4697, 0.4547, 0.4846,
                     0.5677, 0.4784, 0.4777, 0.5017],
                    [0.5542, 0.4820, 0.4521, 0.4912, 0.5167, 0.5015, 0.4697, 0.4547, 0.4846,
                     0.5677, 0.4784, 0.4777, 0.5017]])

In [175]:
t.size()

torch.Size([8, 13])

In [170]:
len([0.5542, 0.4820, 0.4521, 0.4912, 0.5167, 0.5015, 0.4697, 0.4547, 0.4846,
                            0.5677, 0.4784, 0.4777, 0.5017, 0.5542, 0.4820, 0.4521, 0.4912, 0.5167,
                            0.5015, 0.4697, 0.4547, 0.4846, 0.5677, 0.4784, 0.4777, 0.5017, 0.5542,
                            0.4820, 0.4521, 0.4912, 0.5167, 0.5015, 0.4697, 0.4547, 0.4846, 0.5677,
                            0.4784, 0.4777, 0.5017, 0.5542, 0.4820, 0.4521, 0.4912, 0.5167, 0.5015,
                            0.4697, 0.4547, 0.4846, 0.5677, 0.4784, 0.4777, 0.5017, 0.5542, 0.4820,
                            0.4521, 0.4912, 0.5167, 0.5015, 0.4697, 0.4547, 0.4846, 0.5677, 0.4784,
                            0.4777, 0.5017, 0.5542, 0.4820, 0.4521, 0.4912, 0.5167, 0.5015, 0.4697,
                            0.4547, 0.4846, 0.5677, 0.4784, 0.4777, 0.5017, 0.5542, 0.4820, 0.4521,
                            0.4912, 0.5167, 0.5015, 0.4697, 0.4547, 0.4846, 0.5677, 0.4784, 0.4777,
                            0.5017, 0.5542, 0.4820, 0.4521, 0.4912, 0.5167, 0.5015, 0.4697, 0.4547,
                            0.4846, 0.5677, 0.4784, 0.4777, 0.5017])

104

In [171]:
t_true = torch.Tensor([2.0000e-01, 6.0000e-02, 2.0000e-02, 3.0000e-02, 1.0000e+00, 2.0000e+00,
                            5.0000e-02, 2.0000e+00, 5.0000e-01, 5.0000e-01, 2.5000e-01, 2.0000e-02,
                            3.0000e-02,        nan, 1.2000e-01, 5.0000e-01, 6.0000e-02, 5.0000e-01,
                            2.0000e+00,        nan, 4.0000e+00, 5.0000e-01, 1.0000e+00,        nan,
                            1.2000e-01, 3.0000e-02, 2.0000e-01, 6.0000e-02, 2.0000e-02,        nan,
                                   nan, 2.0000e+00,        nan, 4.0000e+00, 2.5000e-01,        nan,
                            1.2000e-01, 2.0000e-02, 3.0000e-02,        nan, 6.0000e-02,        nan,
                            5.0000e-03, 1.0000e+00, 4.0000e+00, 5.0000e-02, 2.0000e+00, 2.5000e-01,
                                   nan, 2.5000e-01, 1.2000e-01, 3.0000e-02, 2.0000e-01,        nan,
                                   nan, 5.0000e-03, 8.0000e+00, 2.0000e+00,        nan, 5.0000e-01,
                                   nan, 2.5000e-01,        nan, 5.0000e-01,        nan, 2.0000e-01,
                            3.0000e-02, 2.0000e-02, 3.0000e-02, 1.0000e+00, 1.0000e+00, 5.0000e-02,
                            2.0000e+00, 4.0000e+00, 5.0000e-01, 4.0000e+00, 1.2000e-01, 3.0000e-02,
                            2.0000e-01, 1.5000e-02, 6.0000e-02, 3.0000e-02,        nan, 4.0000e+00,
                            4.0000e-01, 2.0000e+00, 2.5000e-01,        nan, 2.5000e-01, 1.0000e+01,
                            3.0000e+00, 2.0000e-01, 5.0000e-03, 2.0000e-02, 5.0000e-03, 2.0000e+00,
                            1.0000e+00, 1.6000e+00, 5.0000e-01, 1.0000e+00, 1.2000e-01, 5.0000e-01,
                            5.0000e+00, 1.0000e+00])

104

In [ ]:
#%%
fig, ax = plt.subplots()
x = np.arange(1, epoch+1, 1)
ax.plot(x, train_epoch_loss,label='Training')
ax.plot(x, test_epoch_loss,label='Validation')
ax.legend()
ax.set_xlabel("Number of Epoch")
ax.set_ylabel("Loss")
ax.set_xticks(np.arange(0, epoch+1, 10))
ax.set_title(f'Learning_rate:{lr}')
# ax_2 = ax.twinx()
# ax_2.plot(history["lr"], "k--", lw=1)
# ax_2.set_yscale("log")
# ax.set_ylim(ax.get_ylim()[0], history["training_losses"][0])
ax.grid(axis="x")
fig.tight_layout()
fig.show()
#%%
a = torch.zeros(1, 2, 3, 4, 5, 6)
b = a.view(a.shape[:2], -1, a.shape[5:])
# %%
# from torchviz import make_dot
# x = torch.randn(2, 4, 56).to(device)
# m = model_torch_simple.raw_seq_model().to(device)
# y = m(x)
# make_dot(y, params=dict(list(m.named_parameters()))).render("cnn_torchviz", format="png")